# What is a Partially Signed Bitcoin Transaction?

A Partially Signed Bitcoin Transaction (PSBT) is a transaction interchange format which allows multiple users or devices to work together to sign a transaction.

# Creating a PSBT

Since 0.17, Bitcoin Core has had support for RPC commands for manipulating PSBTs, and we will use these throughout this notebook. The RPCs are exposed as functions on the `node` instance, so it's easy to use them in the test framework.

## Starting up the test node

Run the next cell to start an instance of bitcoind running in regtest mode.

In [ ]:
import util

# Start node
test = util.TestWrapper()
test.setup()
node = test.nodes[0]

print(node.getblockchaininfo()['chain'])

## Creating an address

First, let's create an address to receive at.

In [ ]:
address = node.getnewaddress(address_type="bech32")
print(f'receive address: {address}')

## Getting some coins

Now that the node is up and running and we have a receive address, we need some coins to play with.

In [ ]:
# Generate blocks to create coins. Need to generate 101 blocks due to coinbase maturity rule
txs = node.generatetoaddress(101, address)

In [ ]:
assert(node.getbalance() == 50)
unspent = node.listunspent(1)[0]
inputs = [{"txid": unspent["txid"], "vout": unspent["vout"]}]
print(inputs)

We have some coins, so now let's create another address to send to.

In [ ]:
second_address = node.getnewaddress(address_type="bech32")
print(f'second address: {second_address}')

## Creating the PSBT

Let's create a PSBT which sends 25 of our coins to our second address. The [`walletcreatefundedpsbt`](https://bitcoincore.org/en/doc/0.18.0/rpc/wallet/walletcreatefundedpsbt/) RPC makes that pretty simple.

The arguments are the txid:vout of the input transaction, the address to send to, locktime (zero in this case), and transaction options (fee rate is set to zero for simplicity).


In [ ]:
psbt = node.walletcreatefundedpsbt(inputs, [{ second_address: 25 }])
print(psbt)

## Examining the Signed PSBT

It can be helpful to look at the decoded version of the PSBT, which contains all the information encoded in the PSBT.

In [ ]:
node.decodepsbt(psbt["psbt"])

## Signing the PSBT

We just created the unsigned PSBT, and now we need to sign it. We use the [`walletprocesspsbt`](https://bitcoincore.org/en/doc/0.18.0/rpc/wallet/walletprocesspsbt/) RPC to do that.

In [ ]:
psbt = node.walletprocesspsbt(psbt["psbt"])
print(psbt)

## Analyzing the Signed PSBT

We can also analyze PSBTs with the `analyzepsbt` RPC. A signature has been added to the PSBT, which we can see by the `is_final` property. The next step in the process is `extractor`, so we know a signed transaction can be pulled out of the PSBT as this point.

In [ ]:
node.analyzepsbt(psbt["psbt"])

## Extracting the signed transaction

We have the required number of signatures on the PSBT, so we can extract the final signed transaction.

In [ ]:
tx = node.finalizepsbt(psbt["psbt"])
print(tx)

## Sending the transaction

Finally, the signed transaction can be broadcast, and we can mine a block to recieve the coins on our second address.

In [ ]:
txid = node.sendrawtransaction(tx["hex"])
print(f'tx id: {txid}')

node.generate(1)

The coins have now been received on the second address. This can be verified using the `listunspent` RPC, where there are three unspents now on the wallet. This is expected, since we have the two outputs from our transaction, and one more coinbase maturity due to the block generation.

In [ ]:
node.listunspent()

# Conclusion

In this notebook, we created, signed, and broadcast a simple PSBT. In the next notebook, we'll look at a more complex PSBT involving a multisig address and multiple signers.